In [88]:
import os
import pandas as pd

In [89]:
# %%bash
# git clone https://github.com/Jaehun-Kim22/SCIC_Project.git


In [90]:
## loads excel
xls = pd.ExcelFile('../assets/data.xlsx')
training_data = pd.read_excel(xls, '학습데이터')
priority = pd.read_excel(xls, '우선순위')
data_without_ground_truth = training_data.values[:, 2]

In [91]:
print(training_data, priority)
print(data_without_ground_truth)


       NO    NO2                                                 발화  \
0       1   6438                                   상담직원과 빠른연결했슴좋겠어요   
1       2   6850                                  안기다리고. 빨리상담되었음좋겠다   
2       3   7294                                 너무 오래기다리고 상담이 늦어져요   
3       4   9484                                상담직원과  통화가  매우  힘들다   
4       5  10350                                   상담받으려면오래기다리는게불편함   
..    ...    ...                                                ...   
995  9958    103  카드분실된지 1분만에 카드를 통해서 연락받아 바로 찾았어요 잃어버린지도 몰랐는데 감...   
996  9961      2             수시로 6개월무이자할부와 링크혜택이 다른카드와 구별되는 기능이 편리함   
997  9962      4                               무이자할부 자주 많이 주셔서감사합니다   
998  9963      5                                무이자혜택만족긴할부만족잘쓰고있습니다   
999  9964      7                                 무이자 혜택이 많아 잘쓰고있어요.   

                발화1  발화1의 우선순위         발화2  발화2의 우선순위  발화3  발화3의 우선순위  \
0    불만>고객서비스>상담시스템          5         NaN        NaN  NaN        NaN   
1

In [92]:
## loads excel
# xls = pd.ExcelFile('SCIC_Project/assets/data.xlsx')
# training_data = pd.read_excel(xls, '학습데이터')
# priority = pd.read_excel(xls, '우선순위')
data_without_ground_truth = training_data.values[:, 2]
# !pip install konlpy
from konlpy.tag import Kkma
import copy
import time
kkma = Kkma()


print(training_data.values[:, 2][:5])


def perform_Kkma(text):
  temp_kkma = kkma.sentences(text)
  # if type(text) != type(temp_kkma):
  if [text] != temp_kkma:
    new_string_to_add = ''
    # for string in temp_kkma:
    #   new_string_to_add += string
    new_string_to_add += ' '.join(temp_kkma)
    return new_string_to_add
  else:
    return text
s = time.time()
# print(kkma.sentences(u'잘하는짓이다. 이못난놈아'))
# training_data.info()
training_data_copy_kkma = copy.deepcopy(training_data)


cnt = 0
# column_added_kkma = training_data['발화']
# for text in column_added_kkma:
#   column_added_kkma[cnt] = perform_Kkma(text)
#   cnt += 1

column_added_kkma = []
for text in training_data['발화']:
  column_added_kkma.append(perform_Kkma(text))
  cnt += 1
training_data_copy_kkma['Kkma'] = column_added_kkma
print(training_data_copy_kkma.values[:, 10][:5])
# training_data["Kkma"] = kkma.sentences(u[training_data["발화"]])
e = time.time()
print(e - s)

['상담직원과 빠른연결했슴좋겠어요' '안기다리고. 빨리상담되었음좋겠다' '너무 오래기다리고 상담이 늦어져요'
 '상담직원과  통화가  매우  힘들다' '상담받으려면오래기다리는게불편함']
['상담 직원과 빠른연결했슴좋겠어요' '안 기다리고. 빨리 상담되었음 좋겠다' '너무 오래 기다리고 상담이 늦어져요'
 '상담 직원과 통화가 매우 힘들다' '상담 받으려면 오래 기다리는 게 불편함']
28.25795841217041
